In [3]:
import numpy as np
a = np.matrix([[1,2],[-1,-3]])
b = np.linalg.inv(a)
print(a@b)

[[1. 0.]
 [0. 1.]]


In [ ]:
def get_inv_matrix(frame):
    with open(os.path.join(calib_path, frame+".txt")) as f:
        lines = f.readlines()
        trans = [x for x in filter(lambda s: s.startswith("Tr_velo_to_cam"), lines)][0]
        matrix = [m for m in map(lambda x: float(x), trans.split(" ")[1:])]
        matrix = matrix + [0,0,0,1]
        m = np.array(matrix)
        velo_to_cam  = m.reshape([4,4])


        trans = [x for x in filter(lambda s: s.startswith("R0_rect"), lines)][0]
        matrix = [m for m in map(lambda x: float(x), trans.split(" ")[1:])]        
        m = np.array(matrix).reshape(3,3)
        
        m = np.concatenate((m, np.expand_dims(np.zeros(3), 1)), axis=1)
        
        rect = np.concatenate((m, np.expand_dims(np.array([0,0,0,1]), 0)), axis=0)        
        
        
        m = np.matmul(rect, velo_to_cam)

        
        m = np.linalg.inv(m)
        
        return m


In [ ]:
# def camera_to_lidar(points, r_rect, velo2cam):
#     points_shape = list(points.shape[0:-1])
#     if points.shape[-1] == 3:
#         points = np.concatenate([points, np.ones(points_shape + [1])], axis=-1)
#     lidar_points = points @ np.linalg.inv((r_rect @ velo2cam).T)
#     return lidar_points[..., :3]


# def lidar_to_camera(points, r_rect, velo2cam):
#     points_shape = list(points.shape[:-1])
#     if points.shape[-1] == 3:
#         points = np.concatenate([points, np.ones(points_shape + [1])], axis=-1)
#     camera_points = points @ (r_rect @ velo2cam).T
#     return camera_points[..., :3]


# def box_camera_to_lidar(data, r_rect, velo2cam):
#     xyz = data[:, 0:3]
#     l, h, w = data[:, 3:4], data[:, 4:5], data[:, 5:6]
#     r = data[:, 6:7]
#     xyz_lidar = camera_to_lidar(xyz, r_rect, velo2cam)
#     return np.concatenate([xyz_lidar, w, l, h, r], axis=1)


# def box_lidar_to_camera(data, r_rect, velo2cam):
#     xyz_lidar = data[:, 0:3]
#     w, l, h = data[:, 3:4], data[:, 4:5], data[:, 5:6]
#     r = data[:, 6:7]
#     xyz = lidar_to_camera(xyz_lidar, r_rect, velo2cam)
#     return np.concatenate([xyz, l, h, w, r], axis=1)

In [1]:
#生成txt文件
import math
import json
def vel_to_cam(path, name ='000000.txt'):
    with open(path + name,'r') as f_txt:
        lines = f_txt.readlines()
        trans = [x for x in filter(lambda s: s.startswith("Tr_velo_to_cam"), lines)][0]
        matrix = [m for m in map(lambda x: float(x), trans.split(" ")[1:])]
        matrix = matrix + [0,0,0,1]
        m = np.array(matrix)
        velo_to_cam  = m.reshape([4,4])


        trans = [x for x in filter(lambda s: s.startswith("R0_rect"), lines)][0]
        matrix = [m for m in map(lambda x: float(x), trans.split(" ")[1:])]        
        m = np.array(matrix).reshape(3,3)

        m = np.concatenate((m, np.expand_dims(np.zeros(3), 1)), axis=1)

        rect = np.concatenate((m, np.expand_dims(np.array([0,0,0,1]), 0)), axis=0)        


        m = np.matmul(rect, velo_to_cam)


    #     m = np.linalg.inv(m)

        return m
# m = vel_to_cam(1)
# print(m)
def json_to_txt(name, out_path):
    with open('/home/dk/SUSTechPOINTS/data/my_dataset_all/label/'+name+'.json','r') as f:
        obj= json.load(f)
    m = vel_to_cam('/home/dk/SUSTechPOINTS/data/my_dataset_all/')
    # print(trans)
    for i in obj:
    #     print(np.array(i['psr']['position']))
        pos = np.array([i['psr']['position']['x'], i['psr']['position']['y'],i['psr']['position']['z'] - 
                        i['psr']['scale']['z']/2, 1]).T
    #     pos1 = np.array([i['psr']['position']['x'], i['psr']['position']['y'], i['psr']['position']['z'], 1]).T
    #     print(pos, pos1)
        trans_pos = np.matmul(m,pos)
        height = i['psr']['scale']['z']
        length = i['psr']['scale']['y']
        width = i['psr']['scale']['x']
        rotation_y = -math.pi/2 - i['psr']['rotation']['z']
        #print(trans_pos)
        #print('Car 0 0 0 455.7 183.86 533.81 241.91 ' + str(round(height,2)) +' '+ str(round(length,2)) +' ' + str(round(width,2)) +' '+ str(round(trans_pos[0],2)) +' '+ str(round(trans_pos[1],2)) +' '+ str(round(trans_pos[2],2)) +' '+ str(round(rotation_y,2)))
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        with open(out_path+name+'.txt','a+') as f2:
            f2.writelines('Car 0 0 0 455.7 183.86 533.81 241.91 ' + str(round(height,2)) +' '+ str(round(length,2)) +' ' + str(round(width,2)) +' '+ str(round(trans_pos[0],2)) +' '+ str(round(trans_pos[1],2)) +' '+ str(round(trans_pos[2],2)) +' '+ str(round(rotation_y,2))+'\n')

In [6]:
##查看生成的tst是否正确，对比json
import math
import json
def vel_to_cam(path):
    with open('/home/dk/SUSTechPOINTS/data/my_dataset_all/000000.txt','r') as f_txt:
        lines = f_txt.readlines()
        trans = [x for x in filter(lambda s: s.startswith("Tr_velo_to_cam"), lines)][0]
        matrix = [m for m in map(lambda x: float(x), trans.split(" ")[1:])]
        matrix = matrix + [0,0,0,1]
        m = np.array(matrix)
        velo_to_cam  = m.reshape([4,4])


        trans = [x for x in filter(lambda s: s.startswith("R0_rect"), lines)][0]
        matrix = [m for m in map(lambda x: float(x), trans.split(" ")[1:])]        
        m = np.array(matrix).reshape(3,3)

        m = np.concatenate((m, np.expand_dims(np.zeros(3), 1)), axis=1)

        rect = np.concatenate((m, np.expand_dims(np.array([0,0,0,1]), 0)), axis=0)        


        m = np.matmul(rect, velo_to_cam)


    #     m = np.linalg.inv(m)

        return m

def cam_to_vel(path):
    with open('/home/dk/SUSTechPOINTS/data/my_dataset_all/000000.txt','r') as f_txt:
        lines = f_txt.readlines()
        trans = [x for x in filter(lambda s: s.startswith("Tr_velo_to_cam"), lines)][0]
        matrix = [m for m in map(lambda x: float(x), trans.split(" ")[1:])]
        matrix = matrix + [0,0,0,1]
        m = np.array(matrix)
        velo_to_cam  = m.reshape([4,4])


        trans = [x for x in filter(lambda s: s.startswith("R0_rect"), lines)][0]
        matrix = [m for m in map(lambda x: float(x), trans.split(" ")[1:])]        
        m = np.array(matrix).reshape(3,3)

        m = np.concatenate((m, np.expand_dims(np.zeros(3), 1)), axis=1)

        rect = np.concatenate((m, np.expand_dims(np.array([0,0,0,1]), 0)), axis=0)        


        m = np.matmul(rect, velo_to_cam)


        m = np.linalg.inv(m)

        return m
inv_m = cam_to_vel(1)
with open('/home/dk/SUSTechPOINTS/data/my_dataset_all/label/000006.json','r') as f:
    f1 = f
    print(json.load(f1))
    with open('/home/dk/SUSTechPOINTS/data/my_dataset_all/label_txt/000006.txt','r') as f2:
        l = f2.readlines()[0]
        words = l.strip().split(" ")
#         print(words)
        obj = {}
        pos = np.array([float(words[11]), float(words[12]), float(words[13]), 1]).T
        trans_pos = np.matmul(inv_m, pos)
        #print(trans_pos)

        obj["obj_type"] = words[0]
        obj["psr"] = {"scale": 
                       {"z":float(words[8]),    #height
                         "x":float(words[10]),  #length
                         "y":float(words[9])},  #width
                        "position": {"x":trans_pos[0], "y":trans_pos[1], "z":trans_pos[2]+float(words[8])/2},
                        "rotation": {"x":0, 
                                     "y":0,
                                     "z": -math.pi/2 -float(words[14])}}
        print(obj)

[{'psr': {'position': {'x': 7.906420416239251, 'y': 14.056841648745534, 'z': -0.6361759845167398}, 'scale': {'x': 2.9736764285955974, 'y': 1.606004806541132, 'z': 1.3554377667605877}, 'rotation': {'x': 0, 'y': 0, 'z': -1.597941115041151}}, 'obj_type': 'Car', 'obj_id': ''}]
{'obj_type': 'Car', 'psr': {'scale': {'z': 1.36, 'x': 2.97, 'y': 1.61}, 'position': {'x': 7.907210348574928, 'y': 14.05302234830936, 'z': -0.6309184417423447}, 'rotation': {'x': 0, 'y': 0, 'z': -1.6007963267948966}}}


In [ ]:
import os
path = '/home/dk/SUSTechPOINTS/data/my_dataset_all/label'
out_path = '/home/dk/SUSTechPOINTS/data/my_dataset_all/label_txt/'
list_name = os.listdir(path)
for name in list_name:
    f_name = name.split('.json')[0]
    json_to_txt(f_name, out_path)

In [17]:
def write_txt(out_path = '/home/dk/SUSTechPOINTS/data/my_dataset_all/calib_kitti'):
    with open('/home/dk/SUSTechPOINTS/data/my_dataset_all/000000.txt','r') as f1:
    #     f1_ = f1.read()
    #     print(f1_)
        f1_ = f1.read()
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        for name in range(0,210):
            with open(out_path + '/%06d.txt'%name,'w') as f2:
#                 print(f1_)
                f2.write(f1_)        
def have_write_txt(out_path = '/home/dk/SUSTechPOINTS/data/my_dataset_all/have_calib_kitti/'):
    all_name = os.listdir('/home/dk/SUSTechPOINTS/data/my_dataset_all/label')
    with open('/home/dk/SUSTechPOINTS/data/my_dataset_all/000000.txt','r') as f1:
    #     f1_ = f1.read()
    #     print(f1_)
        f1_ = f1.read()
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        for name in all_name:
            name_ = name.split('.json')[0]
#             print(name_)
            with open(out_path + name_ +'.txt','w') as f2:
#                 print(f1_)
                f2.write(f1_)        

In [18]:
# write_txt()
have_write_txt()

000383
000306
000235
000367
000075
000478
000256
000440
000282
000205
000043
000254
000279
000292
000098
000023
000429
000305
000370
000400
000046
000007
000375
000355
000283
000399
000161
000076
000253
000177
000445
000167
000442
000395
000477
000531
000245
000293
000258
000051
000040
000252
000308
000021
000016
000026
000284
000248
000354
000349
000377
000391
000096
000307
000037
000249
000401
000210
000381
000398
000444
000481
000025
000031
000529
000251
000353
000169
000532
000162
000240
000448
000164
000530
000259
000039
000276
000220
000172
000450
000203
000250
000100
000336
000397
000238
000417
000244
000020
000163
000335
000291
000378
000018
000176
000171
000382
000030
000054
000394
000175
000352
000294
000212
000241
000295
000202
000255
000435
000038
000170
000173
000290
000359
000357
000384
000042
000428
000011
000424
000050
000246
000236
000044
000437
000287
000426
000097
000166
000019
000379
000078
000079
000449
000286
000008
000374
000022
000237
000281
000099
000332
000480

In [20]:
#分训练测试集
import os
path = '/home/dk/SUSTechPOINTS/data/my_dataset_all/label_txt'
f_train = open('/home/dk/SUSTechPOINTS/data/my_dataset_all/train.txt','w')
f_val = open('/home/dk/SUSTechPOINTS/data/my_dataset_all/val.txt','w')
f_trainval = open('/home/dk/SUSTechPOINTS/data/my_dataset_all/train_val.txt','w')
f_test = open('/home/dk/SUSTechPOINTS/data/my_dataset_all/test.txt','w')
enumerate
list_name = os.listdir(path)
for i,name in enumerate(list_name):
    f_name = name.split('.txt')[0]
    f_trainval.write(str(f_name)+'\n')
    f_test.write(str(f_name)+'\n')
    if i % 8 ==0:
        f_val.write(str(f_name)+'\n')
    else:
        f_train.write(str(f_name)+'\n')
#     print(i,f_name)
f_train.close()
f_test.close()
f_trainval.close()
f_val.close()